In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Project

### Summary

### Package List

In [ ]:
#import pkg_resources
#installed_packages = pkg_resources.working_set
#installed_packages_list = sorted(["%s==%s" % (i.key, i.version) for i in installed_packages])
#installed_packages_list

In [ ]:
# Install Additional Packages
import io
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet, LogisticRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import LinearSVR
import xgboost

from scipy import stats #Confidence Interval

### Load Data

In [ ]:
df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
# Get the column names
columns_df1 = df_train.columns
columns_df2 = df_test.columns

# Find the missing column
missing_in_df1 = columns_df2[~columns_df2.isin(columns_df1)]
missing_in_df2 = columns_df1[~columns_df1.isin(columns_df2)]

if not missing_in_df1.empty:
    print(f"Columns missing in df1: {missing_in_df1}")
if not missing_in_df2.empty:
    print(f"Columns missing in df2: {missing_in_df2}")

## Combine Train/Test

In [ ]:
# Train Data
df_train['TRAIN_INDEX'] = 1
#df_train.columns.to_list()
#df_train.shape

# Test Data
df_test['TRAIN_INDEX'] = 0
#df_test.columns.to_list()
#df_test.shape

# Combine Train/Test
df = pd.concat([df_train, df_test], ignore_index=True)
df.shape

## Functions

### Exploratory Data Analysis Functions

In [ ]:
# Code to determine dtypes, missing values, etc...
def analysis(read):
    
    if len(read) > 0:
        print("PROCESS HAS BEEN STARTED\n")

        print("DATA SHAPE")
        print("Observation:", read.shape[0], "Column:", read.shape[1], "\n")

        print("EXPLORE MORE ABOUT THE DATA")
        if len(read.select_dtypes("object").columns) > 0:
            print("Object Variables:", "\n", "Variables:", 
                  len(read.select_dtypes("object").columns), "\n", 
                  read.select_dtypes("object").columns.tolist(), "\n")

        if len(read.select_dtypes("integer").columns) > 0:
            print("Integer Variables:", "\n", "VVariables:", 
                  len(read.select_dtypes("integer").columns), "\n", 
                  read.select_dtypes("integer").columns.tolist(), "\n")

        if len(read.select_dtypes("float").columns) > 0:
            print("Float Variables:", "\n", "Variables:", 
                  len(read.select_dtypes("float").columns), "\n", 
                  read.select_dtypes("float").columns.tolist(), "\n")

        if len(read.select_dtypes("bool").columns) > 0:
            print("Bool Variables:", "\n", "Variables:", 
                  len(read.select_dtypes("bool").columns), "\n", 
                  read.select_dtypes("bool").columns.tolist(), "\n")

        print("IS THERE ANY MISSING VALUE")
        print(" \n ", np.where(read.isnull().values.any() == False,"No missing value!", "Data includes missing value!"), "\n")

        buf = io.StringIO()
        read.info(buf=buf)
        check = True
        check = buf.getvalue().split('\n')[-2].split(":")[1].strip()
        print("MEMORY \n", check)

    else:
        print("ERROR!")

    return read

In [ ]:
# Inspect Missing Values
def data_cleaning(df):

    print("*********{} *********".format('Inspecting missing values'))
    
    data = df.isna().sum().reset_index().sort_values(by=0, ascending=False)
    clean_data = data[data[0] != 0].shape[0]
    columns = df.shape[1]
    rows = df.shape[0]
    data.columns = ["name", "missing appearences"]
    data["%missing from total"] = data[data["missing appearences"]!=0]["missing appearences"]/rows
    mis_data = data[data["%missing from total"] > 0.5].shape[0]
    #drop_data = np.array(data[data["%missing from total"] > 0.5]["name"])
    
    print("{}/{} total missing data in terms of column shape.".format(clean_data, columns))
    #print("{}/{} columns  will be dropped. name of the drop column is {}".format(mis_data, columns,drop_data))
    
    return data#, drop_data

In [ ]:
def histogram_numerical_plots(data):
    data_x = data.copy()
    # Select numerical columns
    num_cols = data_x.select_dtypes(include=['int64', 'float64'])
    # Create subset DataFrame with only numerical values
    data_x_num_cols = data_x[num_cols.columns]
    data_x_num_cols.hist(bins=25, figsize=(40,30))
    plt.show()

In [ ]:
# Pearson's R
def corr_matrix(data):
    data_x = data.copy()
    # Select numerical columns
    num_cols = data_x.select_dtypes(include=['int64', 'float64'])
    corr_cols = data_x[num_cols.columns]
    corr_matrix = corr_cols.corr()
    corr_matrix = corr_matrix["SalePrice"].sort_values(ascending=False)
    return corr_matrix

In [ ]:
df['TRAIN_INDEX'].unique()

In [ ]:
# Univariate Analysis (Compare Train/Test Splits)
def univariate_eda(data, target, var) -> dict:
    if data[var].dtype == 'object':
        data[var] = np.where(data[var].isna(), "NaN", data[var])
        train = data.loc[data['TRAIN_INDEX'] == 1]
        test = data.loc[data['TRAIN_INDEX'] == 0]
        train_grp = train[[var,target]].groupby(var).agg(['count','mean'])
        test_grp = test[[var,target]].groupby(var).agg(['count','mean'])
        grp = pd.merge(train_grp, test_grp, how='outer', left_index=True, right_index=True)
        grp.columns = ['count_train', 'severity_train', 'count_test', 'severity_test']
        print(grp)
        
        count_max = max([max(grp['count_train']),max(grp['count_test'])])
        sev_max = max([max(grp['severity_train']),max(grp['severity_test'])])
        
        df_count = grp[['count_train', 'count_test']].copy()
        df_sev = grp[['severity_train', 'severity_test']].copy()
        
        x = np.arange(len(df_count.index))
        width = 0.3
        multiplier = 0 
        
        fig, ax1 = plt.subplots(constrained_layout=True)
        
        for attribute, measurement in df_count.items():
            offset = width + multiplier
            rects = ax1.bar(x + offset, measurement, width, label=attribute)
            multiplier += 1
            
        ax1.set_title('numerical: ' + var)
        
        ax1.set_ylabel('Counts')
        ax1.set_xticks(x)
        ax1.set_xticklabels(df_count.index, rotation=90)
        ax1.legend(loc='upper left')
        
        ax2 = ax1.twinx()
        ax2.set_ylabel('Severity')
        ax2.plot(df_sev.index, df_sev['severity_train'], c='blue', label='severity_train')
        ax2.plot(df_sev.index, df_sev['severity_test'], c='darkorange', label='severity_test')
        ax2.legend(loc='upper right')
    else:
        data_temp = data.copy()
        data_temp['DECILE'] = pd.qcut(data_temp[var], 10, labels=False, duplicates='drop')                   
        temp = data.copy()
        temp['DECILE'] = pd.qcut(temp[var], 10, duplicates='drop')
        data_grp = temp[['DECILE', target]].groupby('DECILE').agg(['count','mean']).reset_index()
        data_grp.columns = [var+'_BOUNDARY', 'count', 'severity']
        data_grp = data_grp.drop(['count', 'severity'], axis=1)
        
        train = data_temp.loc[data_temp['TRAIN_INDEX'] == 1]
        test = data_temp.loc[data_temp['TRAIN_INDEX'] == 0]
        
        train_temp = train.copy()
        train_grp = train_temp[['DECILE', target]].groupby('DECILE').agg(['count','mean'])
        train_grp.columns = ['count_train', 'severity_train']
        
        test_temp = test.copy()
        test_grp = test_temp[['DECILE', target]].groupby('DECILE').agg(['count','mean'])
        test_grp.columns = ['count_test', 'severity_test']
        
        grp = pd.merge(data_grp, train_grp, how='outer', left_index=True, right_index=True)
        grp = pd.merge(grp, test_grp, how='outer', left_index=True, right_index=True)
        
        df_count = grp[['count_train', 'count_test']].copy()
        df_sev = grp[['severity_train', 'severity_test']].copy()
        
        x = np.arange(len(df_count.index))
        width = 0.3
        multiplier = 0
        
        fig, ax1 = plt.subplots(constrained_layout=True)
        
        for attribute, measurement in df_count.items():
            offset = width * multiplier
            rects = ax1.bar(x + offset, measurement, width, label=attribute)
            multiplier +=1
            
        ax1.set_title('numerical: ' + var)
        
        ax1.set_ylabel('Counts')
        ax1.set_xticks(x)
        ax1.set_xticklabels(df_count.index, rotation=90)
        ax1.legend(loc='upper left')

        ax2 = ax1.twinx()
        ax2.set_ylabel('Severity')
        ax2.plot(df_sev.index, df_sev['severity_train'], label='severity_train')
        ax2.plot(df_sev.index, df_sev['severity_test'], label='severity_test')
        ax2.legend(loc='upper right')

### Modeling Performance Functions

In [ ]:
def evaluation_metrics(actual, pred) -> dict:
    rmse = np.sqrt(np.mean(pred-actual)**2)
    me = np.mean(pred-actual)
    mep = np.mean(pred-actual)/np.mean(actual)
    return {"rmse": Root Mean Squared Error, "me": Mean Error, "mep": Mean Error Percent}

def lift_chart(data, pred, act) -> dict:
    data['Prediction_Decile'] = pd.qcut(data[pred], 20, labels=False, duplicates='drop')
    lift_chart_prediction = data.groupby('Prediction_Decile')[pred].mean().to_frame(name = 'pred').reset_index()
    lift_chart_actual = data.groupby('Prediction_Decile')[act].mean().to_frame(name = 'act').reset_index()
    lift_chart_data = pd.merge(lift_chart_prediction, lift_chart_actual, how='inner', left_on=['Prediction_Decile'], right_on=[])
    
    lift_chart_plot = plt.figsize()
    plt.plot(lift_chart_data['Prediction_Decile'], lift_chart_data['Prediction_Decile'], label = 'pred', color='red')
    plt.plot(lift_chart_data['Prediction_Decile'], lift_chart_data['Prediction_Decile'], label = 'act', color='black')
    plt.xlabel("Prediction Decile")
    plt.ylabel("Average Cost")
    plt.legend(loc="upper right")
    plt.gca().set_title("Model Performance Lift Chart")

In [ ]:
# Understanding Outliers

## Cook's Distance

## Leverage



In [ ]:
# Variable Correlation Analysis

## T-Testing

## Chi-Square Test

from scipy.stats import chi2_contingency

def chi_square_test(df, col1, col2):
    """
    Perform chi-square test of independence between two categorical variables in a DataFrame.

    Parameters:
    df (DataFrame): Pandas DataFrame containing the categorical variables.
    col1 (str): Name of the first categorical column.
    col2 (str): Name of the second categorical column.

    Returns:
    chi2_stat (float): The test statistic of the chi-square test.
    p_val (float): The p-value of the test.
    dof (int): Degrees of freedom.
    contingency_table (DataFrame): The contingency table showing the observed frequencies.
    """

    # Create contingency table
    contingency_table = pd.crosstab(df[col1], df[col2])
    
    # Perform chi-square test
    chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)
    
    return chi2_stat, p_val, dof, contingency_table

## P-Value Testing

## One-Way Analysis

## VIF Calculations

## Correlation Matrix

## Principal Component Analysis (PCA)

## 


In [ ]:
# Execute Analysis, Data Cleaning Functions
output = analysis(df_train)
missing = data_cleaning(df_train)
missing_data = data_cleaning(df_train)
missing_data.head(20)

In [ ]:
### Exploratory Data Analysis
histogram_numerical_plots(df_train)

In [ ]:
### Pearson's R Plot
corr_matrix(df_train)

### Feature Engineering

#### Categorical Variables

In [ ]:
categorical_variables = ['MSZoning', 'Street', 'Alley', 'LotShape'] #, 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 
                         #'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                         #'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                         #'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 
                         #'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 
                         #'SaleType', 'SaleCondition']

In [ ]:
df['LotShape'].unique()

In [ ]:
categorical_variables = ['LotShape']

In [ ]:
for var in categorical_variables:
    univariate_eda(df, 'SalePrice', var)

#### Numerical Variables

In [ ]:
### Imputation Strategies
from sklearn.impute import SimpleImputer

def mean_imputation(data):
    data_x = data.copy()
    mean_imputer = SimpleImputer(strategy="median")
    num_cols = data_x.select_dtypes(include=['int64', 'float64'])
    # Create subset DataFrame with only numerical values
    data_imputation = data_x[num_cols.columns]
    data_x
    
def log_transformation(data):
    
def sqrt_transformation(data):
    


### Data Transformer

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, selected_features):
        self.selected_features = selected_features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.selected_features]

# Define selected features (numerical and categorical)
numerical_features = ['num_feature1', 'num_feature2']
categorical_features = ['cat_feature1', 'cat_feature2']

# Create pipelines for numerical and categorical preprocessing
numerical_pipeline = Pipeline([
    ('num_selector', FeatureSelector(numerical_features)),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('cat_selector', FeatureSelector(categorical_features)),
    ('encoder', OneHotEncoder())
])

# Combine numerical and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])



### Model Pipelines

In [ ]:
## Step-Wise

## Tweedie
glm_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('glm', LinearRegression())  # Replace with your GLM model
])

## Inverse Gaussian

## Logistic/Binomial

## Multinomial


In [ ]:
# GLM pipeline


# Lasso pipeline


### Train/Test Split

In [ ]:
train_data = df[df['TRAIN_IND'] == 1].copy()
test_data = df[df['TRAIN_IND'] == 0].copy()

X_train = train_data.drop(['TRAIN_IND', 'target_column'], axis=1)  # Drop 'TRAIN_IND' and target column
y_train = train_data['target_column']  # Replace 'target_column' with your actual target column

X_test = test_data.drop(['TRAIN_IND', 'target_column'], axis=1)  # Drop 'TRAIN_IND' and target column
y_test = test_data['target_column']  # Replace 'target_column' with your actual target column

### Create Modeling Functions

In [ ]:
# Naive Model

In [ ]:
# Random Forest

In [ ]:
# Decision Tree (CART)

In [ ]:
# Lasso Model
lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('lasso', Lasso(alpha=0.1))  # Replace with your Lasso model and set alpha
])


In [ ]:
# Ridge Model


In [ ]:
# Elastic Net


In [ ]:
# Gradient Boosting Machine


In [ ]:
# Support Vector Machine


In [ ]:
# Neural Network


### Model Fitting

In [ ]:
# Fit GLM pipeline
glm_pipeline.fit(X_train, y_train)

# Evaluate GLM pipeline
glm_score = glm_pipeline.score(X_test, y_test)
print(f"GLM Model R^2 Score: {glm_score}")

# Fit Lasso pipeline
lasso_pipeline.fit(X_train, y_train)

# Evaluate Lasso pipeline
lasso_score = lasso_pipeline.score(X_test, y_test)
print(f"Lasso Model R^2 Score: {lasso_score}")

### Data Analysis